Data Loading with Kaggle

In [2]:
IMAGE_SIZE = 128
NR_OF_EPOCHS = 10

In [4]:
!pip install diffusers

  Using cached diffusers-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.27.2-py3-none-any.whl (2.0 MB)


In [3]:
import torch
from diffusers import DDPMPipeline

# We can set the device to either use our GPU or use our CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda', index=0)

In [4]:
import os
import shutil

# Define the source directory and the target directory
source_dir = 'data0/lsun/bedroom'
target_dir = 'C:/Users/Filip/Desktop/PW/2 semestr/Deep Learning/pro3/content/training'


# Traverse through all subdirectories in the source directory
for root, dirs, files in os.walk(source_dir):
    print(f'Checking directory {root}...')
    for file in files:
        # If the file is an image, move it to the target directory
        if file.endswith('.jpg') or file.endswith('.png'):
            print(f'Moving file {file}...')
            shutil.move(os.path.join(root, file), target_dir)

# Create a zip archive of the target directory
#shutil.make_archive('/content/training_zipped', 'zip', target_dir)

Checking directory data0/lsun/bedroom...
Checking directory data0/lsun/bedroom\0...
Checking directory data0/lsun/bedroom\0\0...
Checking directory data0/lsun/bedroom\0\0\0...
Checking directory data0/lsun/bedroom\0\0\1...
Checking directory data0/lsun/bedroom\0\0\2...
Checking directory data0/lsun/bedroom\0\0\3...
Checking directory data0/lsun/bedroom\0\0\4...
Checking directory data0/lsun/bedroom\0\0\5...
Checking directory data0/lsun/bedroom\0\0\6...
Checking directory data0/lsun/bedroom\0\0\7...
Checking directory data0/lsun/bedroom\0\0\8...
Checking directory data0/lsun/bedroom\0\0\9...
Checking directory data0/lsun/bedroom\0\0\a...
Checking directory data0/lsun/bedroom\0\0\b...
Checking directory data0/lsun/bedroom\0\0\c...
Checking directory data0/lsun/bedroom\0\0\d...
Checking directory data0/lsun/bedroom\0\0\e...
Checking directory data0/lsun/bedroom\0\0\f...
Checking directory data0/lsun/bedroom\0\1...
Checking directory data0/lsun/bedroom\0\1\0...
Checking directory data0/ls

In [4]:
import os
import shutil

# Define the source directory and the target directory
source_dir = '/content/training/'
target_dir = '/content/sample999/'

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Initialize a counter for the number of images moved
count = 0

# Traverse through all subdirectories in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # If the file is an image, move it to the target directory
        if file.endswith('.jpg') or file.endswith('.png'):
            shutil.move(os.path.join(root, file), target_dir)
            count += 1
            # If we've moved 1000 images, stop
            if count >= 10000000:
                break
    # If we've moved 1000 images, stop
    if count >= 10000000:
        break

In [5]:
import os

DIR = '/content/sample999/'
print( len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

263125


Checking for small sample - 89 images

What data augemntataion we can apply, if that even will be changing

In [6]:
from torchvision import transforms

image_size = IMAGE_SIZE

# Define data augmentations
preprocess = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),  # Resize
        transforms.RandomHorizontalFlip(),  # Randomly flip (data augmentation)
        transforms.ToTensor(),  # Convert to tensor (0, 1)
        transforms.Normalize([0.5], [0.5]),  # Map to (-1, 1)
    ]
)

In [7]:
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor
from PIL import Image
import os

batch_size = 16


class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Define the directory where your images are stored
image_dir = '/content/sample999/'

# Define the transformation to preprocess images and convert them to tensors
transform = preprocess

# Load the images into a dataset
dataset = CustomImageDataset(image_dir, transform=transform)


train_dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True
)

In [8]:
#Check if images are loading correctly ADD displaying images
batch = next(iter(train_dataloader))

In [10]:
from diffusers import DDPMScheduler

# We'll learn about beta_start and beta_end in the next sections
scheduler = DDPMScheduler(
    num_train_timesteps=1000, beta_start=0.001, beta_end=0.02
)
timesteps = torch.linspace(0, 999, 8).long()

# We load 8 images from the dataset and
# add increasing amounts of noise to them
x = batch[:8]
noise = torch.rand_like(x)
noised_x = scheduler.add_noise(x, noise, timesteps)

In [11]:
noised_x

tensor([[[[-2.5260e-01, -2.6636e-01, -2.4778e-01,  ..., -4.4520e-01,
           -4.8010e-01, -4.8186e-01],
          [-2.2657e-01, -2.4174e-01, -2.6232e-01,  ..., -4.7081e-01,
           -4.8687e-01, -4.6114e-01],
          [-2.4533e-01, -2.6055e-01, -2.7217e-01,  ..., -4.6390e-01,
           -4.8291e-01, -4.8978e-01],
          ...,
          [-3.0042e-01, -2.7832e-01, -2.0168e-01,  ..., -1.1980e-01,
            3.8492e-02, -5.4251e-02],
          [-2.9435e-01, -2.0817e-01, -1.6995e-01,  ..., -2.9744e-02,
           -4.8673e-02, -1.2496e-01],
          [-2.5134e-01, -2.1936e-01, -1.8143e-01,  ...,  1.5513e-02,
           -9.8994e-02, -1.3088e-01]],

         [[-4.7084e-01, -4.8791e-01, -4.9199e-01,  ..., -6.8625e-01,
           -6.8129e-01, -7.0331e-01],
          [-4.6134e-01, -4.6489e-01, -4.6839e-01,  ..., -6.9959e-01,
           -6.9144e-01, -6.9137e-01],
          [-4.5170e-01, -4.6403e-01, -4.8420e-01,  ..., -6.8458e-01,
           -6.9649e-01, -7.1242e-01],
          ...,
     

In [12]:
import numpy as np
import matplotlib.pyplot as plt
def show_images(noised_x):
    noised_x = noised_x.detach().cpu().numpy()
    noised_x = np.transpose(noised_x, (0, 2, 3, 1))  # reshape from (batch, channels, height, width) to (batch, height, width, channels)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(np.concatenate([np.pad(img, ((2, 2), (2, 2), (0, 0)), mode='constant', constant_values=1)
                              for img in noised_x], axis=1))

In [21]:
#show_images(batch[:8])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 

In [1]:
#show_images(noised_x)

NameError: name 'show_images' is not defined

In [13]:
from diffusers import UNet2DModel

# Create a UNet2DModel
model = UNet2DModel(
    in_channels=3,  # 3 channels for RGB images
    sample_size=64,  # Specify our input size
    # The number of channels per block affects the model size
    block_out_channels=(64, 128, 256, 512),
    down_block_types=(
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",
        "AttnDownBlock2D",
    ),
    up_block_types=("AttnUpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D"),
).to(device)

# Pass a batch of data through to see it works
with torch.no_grad():
    out = model(noised_x.to(device), timestep=timesteps.to(device)).sample

out.shape

c:\Users\Filip\anaconda3\Lib\site-packages\diffusers\models\attention_processor.py:1279: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  hidden_states = F.scaled_dot_product_attention(


torch.Size([8, 3, 128, 128])

In [14]:
from torch.nn import functional as F
from tqdm import tqdm

num_epochs = NR_OF_EPOCHS  # How many runs through the data should we do?
lr = 1e-4  # What learning rate should we use
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
losses = []  # Somewhere to store the loss values for later plotting

# Train the model (this takes a while!)
for epoch in range(num_epochs):
    # Wrap the dataloader with tqdm for a progress bar
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch in progress_bar:
        # Load the input images
        clean_images = batch.to(device)

        # Sample noise to add to the images
        noise = torch.randn(clean_images.shape).to(device)

        # Sample a random timestep for each image
        timesteps = torch.randint(
            0,
            scheduler.config.num_train_timesteps,
            (clean_images.shape[0],),
            device=device,
        ).long()

        # Add noise to the clean images according
        # to the noise magnitude at each timestep
        noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

        # Get the model prediction for the noise
        # The model also uses the timestep as an input
        # for additional conditioning
        noise_pred = model(noisy_images, timesteps, return_dict=False)[0]

        # Compare the prediction with the actual noise
        loss = F.mse_loss(noise_pred, noise)

        # Store the loss for later plotting
        losses.append(loss.item())

        # Update the model parameters with the optimizer based on this loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Update the progress bar with the current loss value
        progress_bar.set_postfix({'loss': loss.item()})


Epoch 10/10: 100%|██████████| 16446/16446 [6:37:18<00:00,  1.45s/it, loss=0.00538]  


In [15]:
torch.save(model.state_dict(), 'C:/Users/Filip/Desktop/PW/2 semestr/Deep Learning/pro3/modelFULL128x128.pt')

In [16]:
from matplotlib import pyplot as plt

plt.subplots(1, 2, figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(losses)
plt.title("Training loss")
plt.xlabel("Training step")

plt.subplot(1, 2, 2)
plt.plot(range(400, len(losses)), losses[400:])
plt.title("Training loss from step 400")
plt.xlabel("Training step");


: 

In [20]:
pipeline = DDPMPipeline(unet=model, scheduler=scheduler)
ims = pipeline(batch_size=4).images
ims
#show_images(ims, nrows=1)

  0%|          | 0/1000 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=64x64>,
 <PIL.Image.Image image mode=RGB size=64x64>,
 <PIL.Image.Image image mode=RGB size=64x64>,
 <PIL.Image.Image image mode=RGB size=64x64>]

In [21]:
import matplotlib.pyplot as plt

def show_images_PIL(images):
    fig, axs = plt.subplots(1, len(images), figsize=(20, 20))
    for i, img in enumerate(images):
        axs[i].imshow(img)
        axs[i].axis('off')

In [1]:
# Convert the tensors back to PIL Images
#images = [transforms.ToPILImage()(img) for img in ims]

# Display the images
show_images_PIL(ims)

NameError: name 'show_images_PIL' is not defined